Import


In [3]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt


création de la dataframe

In [4]:
df = pd.read_csv(r"After_Sort.csv", parse_dates=["BaseDateTime"])
df.sort_values("BaseDateTime", inplace=True)

print informations

In [5]:
print(df.head())
print(df.info())
print(df.describe())
print(df["VesselType"].value_counts())

             id       MMSI BaseDateTime       LAT       LON   SOG    COG  \
117352  9030257  563020500   2023-05-25  28.82655 -89.43528  14.2  207.9   
117395  9032593  366954420   2023-05-25  27.77639 -82.63030   0.1  294.9   
117339  9029199  215583000   2023-05-25  26.48967 -79.39617  10.2  182.7   
117360  9030460  367181550   2023-05-25  29.85450 -89.98148   0.0  298.2   
117442  9036191  368045410   2023-05-25  27.84124 -97.06945   5.9   15.5   

        Heading        VesselName            IMO CallSign  VesselType  Status  \
117352    206.0       MERCURY SKY     IMO9796949   9V5148          70       0   
117395    294.0  PROVINCETOWN III     IMO9329394  WDB8185          60       0   
117339    184.0         COBIA LNG     IMO9869306  9HA5192          84       0   
117360      NaN   BELLE CHASSE II            NaN  WDD7069          60       0   
117442      NaN    JOSEPH F WEBER  IMO1012865430  WDK3453          60       0   

        Length  Width  Draft  Cargo TransceiverClass  
1

ajout des colones lat et long +5, +10, +15 et delta_temps à la dataframe

In [6]:
nouvelles_colonnes = ['LONG_5', 'LAT_5', 'LONG_10', 'LAT_10', 'LONG_15', 'LAT_15','Delta_temps']

# Ajout des colonnes avec des valeurs vides (NaN)
for col in nouvelles_colonnes:
    df[col] = None  # ou pd.NA, ou np.nan si vous utilisez numpy


print(df.head())

             id       MMSI BaseDateTime       LAT       LON   SOG    COG  \
117352  9030257  563020500   2023-05-25  28.82655 -89.43528  14.2  207.9   
117395  9032593  366954420   2023-05-25  27.77639 -82.63030   0.1  294.9   
117339  9029199  215583000   2023-05-25  26.48967 -79.39617  10.2  182.7   
117360  9030460  367181550   2023-05-25  29.85450 -89.98148   0.0  298.2   
117442  9036191  368045410   2023-05-25  27.84124 -97.06945   5.9   15.5   

        Heading        VesselName            IMO  ... Draft  Cargo  \
117352    206.0       MERCURY SKY     IMO9796949  ...   NaN   70.0   
117395    294.0  PROVINCETOWN III     IMO9329394  ...   1.9   60.0   
117339    184.0         COBIA LNG     IMO9869306  ...   9.4   84.0   
117360      NaN   BELLE CHASSE II            NaN  ...   NaN   60.0   
117442      NaN    JOSEPH F WEBER  IMO1012865430  ...   3.3   60.0   

        TransceiverClass  LONG_5  LAT_5  LONG_10  LAT_10 LONG_15 LAT_15  \
117352                 A    None   None     Non

Remplissage de la colonne Delta_temps

In [7]:
df['Delta_temps'] = df.sort_values(['MMSI', 'BaseDateTime']).groupby('MMSI')['BaseDateTime'].diff().dt.total_seconds()

Séparation en bases d'apprentissage, test et validation par MMSI

In [8]:

# Récupération des MMSI uniques et suppr valeurs manquantes
mmsi_uniques = df['MMSI'].dropna().unique()

# Mélange aléatoire
np.random.seed(42)
np.random.shuffle(mmsi_uniques)

# Affectation fixe : 97 pour apprentissage, 30 pour test, 22 pour validation
n_train = 97
n_test = 30
n_val = 22

mmsi_train = mmsi_uniques[:n_train]
mmsi_test = mmsi_uniques[n_train:n_train + n_test]
mmsi_val = mmsi_uniques[n_train + n_test:n_train + n_test + n_val]

# Création des trois sous-ensembles
df_train = df[df['MMSI'].isin(mmsi_train)]
df_test = df[df['MMSI'].isin(mmsi_test)]
df_val = df[df['MMSI'].isin(mmsi_val)]

# Affichage pour vérification
print("Nombre de MMSI uniques:")
print("Train:", df_train['MMSI'].nunique())
print("Test :", df_test['MMSI'].nunique())
print("Val  :", df_val['MMSI'].nunique())




Nombre de MMSI uniques:
Train: 97
Test : 30
Val  : 22


Observation de début de chaque df

In [9]:
print(df_train.head(5))
print("\n")
print("\n")
print("  ===============================================================  ")
print("\n")
print("\n")
print(df_test.head(5))
print("\n")
print("\n")
print("  ===============================================================  ")
print("\n")
print("\n")
print(df_val.head(5))

             id       MMSI        BaseDateTime       LAT       LON   SOG  \
117352  9030257  563020500 2023-05-25 00:00:00  28.82655 -89.43528  14.2   
117442  9036191  368045410 2023-05-25 00:00:00  27.84124 -97.06945   5.9   
117410  9033545  367761740 2023-05-25 00:00:01  29.80445 -93.95542   0.0   
117338  9029195  563142600 2023-05-25 00:00:01  28.90024 -93.96424  12.6   
117341  9029417  366957490 2023-05-25 00:00:01  29.28540 -89.36020   0.0   

          COG  Heading         VesselName            IMO  ... Draft  Cargo  \
117352  207.9    206.0        MERCURY SKY     IMO9796949  ...   NaN   70.0   
117442   15.5      NaN     JOSEPH F WEBER  IMO1012865430  ...   3.3   60.0   
117410    NaN      NaN     GENERAL PATTON            NaN  ...   NaN   33.0   
117338  305.1    306.0        GRAMPUS ACE     IMO9584190  ...   8.7   70.0   
117341  315.3      NaN  WILLIAM H JOHNSON            NaN  ...   NaN   50.0   

        TransceiverClass  LONG_5  LAT_5  LONG_10  LAT_10 LONG_15 LAT_15  \

In [10]:
df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)
print(df_train.head(10))


             id       MMSI        BaseDateTime       LAT       LON   SOG  \
117354  9030354  209513000 2023-05-25 00:00:04  26.34658 -78.94057  31.9   
117425  9035334  209513000 2023-05-25 00:01:06  26.33986 -78.94755  31.9   
117521  9042882  209513000 2023-05-25 00:02:16  26.33224 -78.95551  32.2   
118718  9138497  209513000 2023-05-25 00:11:58  26.26969 -79.02179  31.6   
118261  9101604  209513000 2023-05-25 00:14:42  26.25213 -79.04084  32.3   
119115  9170094  209513000 2023-05-25 00:16:20  26.24117 -79.05147  32.3   
121742  9370175  209513000 2023-05-25 00:18:36  26.22595 -79.06683  32.9   
118618  9129883  209513000 2023-05-25 00:19:42  26.21860 -79.07424  32.5   
118772  9142300  209513000 2023-05-25 00:20:43  26.21183 -79.08091  31.0   
118884  9152712  209513000 2023-05-25 00:21:44  26.20550 -79.08769  32.3   

          COG  Heading   VesselName         IMO  ... Draft  Cargo  \
117354  222.8    224.0  OCEAN PEARL  IMO9060247  ...   9.8   49.0   
117425  224.1    226.0  O

C:\Users\Paul\AppData\Local\Temp\ipykernel_14704\703263359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)


In [11]:
print(f"Taille de l'ensemble d'apprentissage : {len(df_train)}")
print(f"Taille de l'ensemble de test : {len(df_test)}")
print(f"Taille de l'ensemble de validation : {len(df_val)}")


Taille de l'ensemble d'apprentissage : 270763
Taille de l'ensemble de test : 82909
Taille de l'ensemble de validation : 58051


In [12]:
df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)
print(df_train.head())
# print(df_train.iloc[2950:3000])


             id       MMSI        BaseDateTime       LAT       LON   SOG  \
117354  9030354  209513000 2023-05-25 00:00:04  26.34658 -78.94057  31.9   
117425  9035334  209513000 2023-05-25 00:01:06  26.33986 -78.94755  31.9   
117521  9042882  209513000 2023-05-25 00:02:16  26.33224 -78.95551  32.2   
118718  9138497  209513000 2023-05-25 00:11:58  26.26969 -79.02179  31.6   
118261  9101604  209513000 2023-05-25 00:14:42  26.25213 -79.04084  32.3   

          COG  Heading   VesselName         IMO  ... Draft  Cargo  \
117354  222.8    224.0  OCEAN PEARL  IMO9060247  ...   9.8   49.0   
117425  224.1    226.0  OCEAN PEARL  IMO9060247  ...   9.8   49.0   
117521  223.0    224.0  OCEAN PEARL  IMO9060247  ...   9.8   49.0   
118718  224.3    225.0  OCEAN PEARL  IMO9060247  ...   9.8   49.0   
118261  222.3    223.0  OCEAN PEARL  IMO9060247  ...   9.8   49.0   

        TransceiverClass  LONG_5  LAT_5  LONG_10  LAT_10 LONG_15 LAT_15  \
117354                 A    None   None     None    N

C:\Users\Paul\AppData\Local\Temp\ipykernel_14704\1592785229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)


prendre entre 295 et 305sec
prendre entre 595 et 605sec
prendre entre 895 et 905sec

In [13]:
def interpoler_positions_dataframe(df):
    """
    Fonction pour interpoler les positions à 5, 10 et 15 minutes
    pour un dataframe donné
    """
    # Créer une copie pour éviter les warnings
    df_copy = df.copy()
    
    # Trier par MMSI et datetime pour assurer l'ordre chronologique
    df_copy = df_copy.sort_values(['MMSI', 'BaseDateTime']).reset_index(drop=True)
    
    print(f"Début de l'interpolation pour {len(df_copy)} lignes...")
    
    # Parcourir chaque navire
    for mmsi in df_copy['MMSI'].unique():
        # Filtrer les données du navire actuel
        mask_navire = df_copy['MMSI'] == mmsi
        indices_navire = df_copy[mask_navire].index.tolist()
        
        # Parcourir les positions de ce navire
        for i in range(len(indices_navire) - 1):
            idx_actuel = indices_navire[i]
            idx_suivant = indices_navire[i + 1]
            
            # Récupérer les positions et temps
            lat_A = df_copy.loc[idx_actuel, 'LAT']
            lon_A = df_copy.loc[idx_actuel, 'LON']
            time_A = df_copy.loc[idx_actuel, 'BaseDateTime']
            
            lat_B = df_copy.loc[idx_suivant, 'LAT']
            lon_B = df_copy.loc[idx_suivant, 'LON']
            time_B = df_copy.loc[idx_suivant, 'BaseDateTime']
            
            # Calculer l'intervalle de temps en secondes
            delta_temps = (time_B - time_A).total_seconds()
            
            # Interpolation à 5 minutes (300s)
            if delta_temps >= 300:
                fraction_5min = 300 / delta_temps
                lat_5min = lat_A + fraction_5min * (lat_B - lat_A)
                lon_5min = lon_A + fraction_5min * (lon_B - lon_A)
                
                df_copy.loc[idx_actuel, 'LAT_5'] = lat_5min
                df_copy.loc[idx_actuel, 'LONG_5'] = lon_5min
            
            # Interpolation à 10 minutes (600s)
            if delta_temps >= 600:
                fraction_10min = 600 / delta_temps
                lat_10min = lat_A + fraction_10min * (lat_B - lat_A)
                lon_10min = lon_A + fraction_10min * (lon_B - lon_A)
                
                df_copy.loc[idx_actuel, 'LAT_10'] = lat_10min
                df_copy.loc[idx_actuel, 'LONG_10'] = lon_10min
            
            # Interpolation à 15 minutes (900s)
            if delta_temps >= 900:
                fraction_15min = 900 / delta_temps
                lat_15min = lat_A + fraction_15min * (lat_B - lat_A)
                lon_15min = lon_A + fraction_15min * (lon_B - lon_A)
                
                df_copy.loc[idx_actuel, 'LAT_15'] = lat_15min
                df_copy.loc[idx_actuel, 'LONG_15'] = lon_15min
    
    print("Interpolation terminée !")
    return df_copy

# Application de l'interpolation sur les trois ensembles
print("=== INTERPOLATION DE L'ENSEMBLE D'APPRENTISSAGE ===")
df_train_interpolated = interpoler_positions_dataframe(df_train)

print("\n=== INTERPOLATION DE L'ENSEMBLE DE TEST ===")
df_test_interpolated = interpoler_positions_dataframe(df_test)

print("\n=== INTERPOLATION DE L'ENSEMBLE DE VALIDATION ===")
df_val_interpolated = interpoler_positions_dataframe(df_val)

=== INTERPOLATION DE L'ENSEMBLE D'APPRENTISSAGE ===
Début de l'interpolation pour 270763 lignes...
Interpolation terminée !

=== INTERPOLATION DE L'ENSEMBLE DE TEST ===
Début de l'interpolation pour 82909 lignes...
Interpolation terminée !

=== INTERPOLATION DE L'ENSEMBLE DE VALIDATION ===
Début de l'interpolation pour 58051 lignes...
Interpolation terminée !


In [14]:
def verifier_interpolation(df, nom_ensemble):
    """
    Fonction pour vérifier les résultats de l'interpolation
    """
    print(f"\n=== VÉRIFICATION - {nom_ensemble} ===")
    
    # Compter les valeurs non nulles pour chaque colonne d'interpolation
    interpolations_5min = df['LAT_5'].notna().sum()
    interpolations_10min = df['LAT_10'].notna().sum()
    interpolations_15min = df['LAT_15'].notna().sum()
    
    print(f"Interpolations à 5 minutes : {interpolations_5min}")
    print(f"Interpolations à 10 minutes : {interpolations_10min}")
    print(f"Interpolations à 15 minutes : {interpolations_15min}")
    
    # Afficher quelques exemples d'interpolation
    exemples = df[df['LAT_5'].notna()].head(3)
    if len(exemples) > 0:
        print(f"\nExemples d'interpolation à 5 minutes :")
        for idx, row in exemples.iterrows():
            print(f"MMSI {row['MMSI']} : Position originale ({row['LAT']:.5f}, {row['LON']:.5f}) -> Position à +5min ({row['LAT_5']:.5f}, {row['LONG_5']:.5f})")

# Vérification des trois ensembles
verifier_interpolation(df_train_interpolated, "TRAIN")
verifier_interpolation(df_test_interpolated, "TEST")
verifier_interpolation(df_val_interpolated, "VALIDATION")

print("\n=== INTERPOLATION TERMINÉE POUR TOUS LES ENSEMBLES ===")
print("Les DataFrames df_train_interpolated, df_test_interpolated et df_val_interpolated")
print("contiennent maintenant les positions interpolées dans les colonnes LAT_5, LONG_5, LAT_10, LONG_10, LAT_15, LONG_15")


=== VÉRIFICATION - TRAIN ===
Interpolations à 5 minutes : 7218
Interpolations à 10 minutes : 1910
Interpolations à 15 minutes : 1299

Exemples d'interpolation à 5 minutes :
MMSI 209513000 : Position originale (26.33224, -78.95551) -> Position à +5min (26.30000, -78.98967)
MMSI 209513000 : Position originale (26.09208, -80.11716) -> Position à +5min (26.09205, -80.11715)
MMSI 209513000 : Position originale (26.08808, -80.11620) -> Position à +5min (26.08809, -80.11616)

=== VÉRIFICATION - TEST ===
Interpolations à 5 minutes : 2028
Interpolations à 10 minutes : 486
Interpolations à 15 minutes : 348

Exemples d'interpolation à 5 minutes :
MMSI 205776000 : Position originale (25.95847, -97.37876) -> Position à +5min (25.95846, -97.37879)
MMSI 205776000 : Position originale (25.95847, -97.37880) -> Position à +5min (25.95847, -97.37880)
MMSI 205776000 : Position originale (25.95846, -97.37878) -> Position à +5min (25.95844, -97.37879)

=== VÉRIFICATION - VALIDATION ===
Interpolations à 5 m

In [15]:
seuil_SOG = 0.5  # à ajuster selon ton dataset
df_filtre = df[df['SOG'] > seuil_SOG].reset_index(drop=True)
len(df_filtre)

132876

1. Filtrage des bateaux à l’arrêt et préparation des données

In [26]:
# Filtrer les bateaux à l'arrêt
seuil_SOG = 0.5  # à ajuster selon ton dataset
df_filtre = df[df['SOG'] > seuil_SOG].reset_index(drop=True)

# Supprimer toutes les lignes où il manque COG ou Heading
df_filtre = df_filtre.dropna(subset=['COG', 'Heading']).reset_index(drop=True)

# Garder uniquement les lignes où on a la cible (LAT_5 et LONG_5)
df_lstm = df_filtre.dropna(subset=['LAT_5', 'LONG_5']).reset_index(drop=True)

# Définir les features d'entrée et la cible
features = ['LAT', 'LON', 'SOG', 'COG', 'Heading']  # tu peux adapter si besoin
target = ['LAT_5', 'LONG_5']
seq_len = 10  # ajuste si besoin

def create_lstm_sequences(df, features, target, seq_len):
    X, y = [], []
    for i in range(len(df) - seq_len):
        X.append(df.loc[i:i+seq_len-1, features].values)
        y.append(df.loc[i+seq_len, target].values)
    return np.array(X), np.array(y)

X, y = create_lstm_sequences(df_lstm, features, target, seq_len)
print("Shape X:", X.shape, "Shape y:", y.shape)


Shape X: (0,) Shape y: (0,)


LSTMoooooooooooooooooooooooooooooooooooooo

In [17]:
seq_len = 20
features = ['LAT', 'LON', 'SOG', 'COG', 'Heading']  # adapte si besoin
target = ['LAT_5', 'LONG_5']

In [18]:
def create_lstm_sequences(df, features, target, seq_len):
    X, y = [], []
    df = df.dropna(subset=features + target).reset_index(drop=True)
    for i in range(len(df) - seq_len):
        X.append(df.loc[i:i+seq_len-1, features].values)
        y.append(df.loc[i+seq_len, target].values)
    return np.array(X), np.array(y)

In [19]:
X_train, y_train = create_lstm_sequences(df_train, features, target, seq_len)
X_val, y_val = create_lstm_sequences(df_val, features, target, seq_len)
X_test, y_test = create_lstm_sequences(df_test, features, target, seq_len)

print("Shape X_train:", X_train.shape, "Shape y_train:", y_train.shape)


Shape X_train: (0,) Shape y_train: (0,)


In [20]:
model = keras.Sequential([
    layers.Input(shape=(seq_len, len(features))),
    layers.LSTM(64, return_sequences=False),
    layers.Dense(32, activation='relu'),
    layers.Dense(2)  # 2 sorties : LAT_5, LONG_5
])

model.compile(optimizer='adam', loss='mae', metrics=['mae'])

In [22]:
print(X_train.shape)
print(y_train.shape)

# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_val, y_val),
#     epochs=15,
#     batch_size=128,
#     verbose=1
# )

(0,)
(0,)


In [ ]:
mae_val = model.evaluate(X_val, y_val, verbose=0)[1]
mae_test = model.evaluate(X_test, y_test, verbose=0)[1]
print(f"MAE Validation : {mae_val:.6f}")
print(f"MAE Test       : {mae_test:.6f}")

In [ ]:
y_pred = model.predict(X_test)

plt.figure(figsize=(12, 6))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.plot([lat for lat in X_test[i,:,0]], [lon for lon in X_test[i,:,1]], 'bo-', label='Historique')
    plt.plot(y_test[i,0], y_test[i,1], 'go', label='Réel +5min')
    plt.plot(y_pred[i,0], y_pred[i,1], 'rx', label='Prévu +5min')
    plt.title(f"Exemple {i+1}")
    if i == 0:
        plt.legend()
plt.suptitle("Trajectoires : Historique (bleu), Réel +5min (vert), Prévu +5min (rouge)")
plt.tight_layout()
plt.show()